In [1]:
import random
from string import ascii_letters

import pandas as pd

random.seed(3.14)


# 数据库风格的聚合方式

In [2]:
x = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "x": ["x1", "x2", "x3"],
    }
)
x.head()

,id,x
0,1,x1
1,2,x2
2,3,x3


In [3]:

y = pd.DataFrame(
    {
        "id": [1, 2, 3, 3, 4],
        "y": ["y1", "y3", "y4", "y5", "y6"],
    }
)
y.head()

,id,y
0,1,y1
1,2,y3
2,3,y4
3,3,y5
4,4,y6


In [4]:
x.merge(y, on="id")

,id,x,y
0,1,x1,y1
1,2,x2,y3
2,3,x3,y4
3,3,x3,y5


In [5]:
# default join way
x.merge(y, on="id", how="inner")

,id,x,y
0,1,x1,y1
1,2,x2,y3
2,3,x3,y4
3,3,x3,y5


In [6]:
x.merge(y, on="id", how="outer")

,id,x,y
0,1,x1,y1
1,2,x2,y3
2,3,x3,y4
3,3,x3,y5
4,4,NaN,y6


In [7]:
x.merge(y, on="id", how="left")

,id,x,y
0,1,x1,y1
1,2,x2,y3
2,3,x3,y4
3,3,x3,y5


In [8]:
x.merge(y, on="id", how="right")

,id,x,y
0,1,x1,y1
1,2,x2,y3
2,3,x3,y4
3,3,x3,y5
4,4,NaN,y6


#### 关联条件

In [9]:
x["uid"] = [10, 23, 40]
x["gender"] = ["male", "female", "male"]

y["uid"] = [10, 10, 41, 43, 23]
y["sex"] = ["male", None, None, "female", "female"]

In [10]:
x.head()

,id,x,uid,gender
0,1,x1,10,male
1,2,x2,23,female
2,3,x3,40,male


In [11]:
y.head()

,id,y,uid,sex
0,1,y1,10,male
1,2,y3,10,None
2,3,y4,41,None
3,3,y5,43,female
4,4,y6,23,female


In [12]:
x.merge(y, on="id")

,id,x,uid_x,gender,y,uid_y,sex
0,1,x1,10,male,y1,10,male
1,2,x2,23,female,y3,10,None
2,3,x3,40,male,y4,41,None
3,3,x3,40,male,y5,43,female


In [13]:
x.merge(y, on=["id", "uid"])

,id,x,uid,gender,y,sex
0,1,x1,10,male,y1,male


In [14]:
x = x.rename(columns={"id": "id_x"})
x.head()

,id_x,x,uid,gender
0,1,x1,10,male
1,2,x2,23,female
2,3,x3,40,male


In [15]:
x.merge(y, left_on="id_x", right_on="id")

,id_x,x,uid_x,gender,id,y,uid_y,sex
0,1,x1,10,male,1,y1,10,male
1,2,x2,23,female,2,y3,10,None
2,3,x3,40,male,3,y4,41,None
3,3,x3,40,male,3,y5,43,female


In [16]:
x.merge(y, left_on=["id_x", "gender"], right_on=["id", "sex"], how="left")

,id_x,x,uid_x,gender,id,y,uid_y,sex
0,1,x1,10,male,1.0,y1,10.0,male
1,2,x2,23,female,NaN,NaN,NaN,NaN
2,3,x3,40,male,NaN,NaN,NaN,NaN


### 类型不一致问题

In [17]:
left = pd.DataFrame(
    {
        "a": [1, 3, 5, 6, 7],
        "b": random.choices(ascii_letters, k=5),
        "datetime": pd.date_range("20210101", periods=5, freq="D"),
    }
)
left.head()

,a,b,datetime
0,1,O,2021-01-01
1,3,L,2021-01-02
2,5,k,2021-01-03
3,6,F,2021-01-04
4,7,Z,2021-01-05


In [18]:

right = pd.DataFrame(
    {
        "a": ["1", "3", "5", "7", "9"],
        "c": random.choices(ascii_letters, k=5),
        "datetime": pd.date_range("20210103", periods=5, freq="D").map(lambda v: str(v.date())),
    }
)
right.head()

,a,c,datetime
0,1,u,2021-01-03
1,3,t,2021-01-04
2,5,V,2021-01-05
3,7,x,2021-01-06
4,9,m,2021-01-07


In [19]:
# error
# `a` in left is integer type, but in right is string.

# left.merge(right, on=["a"])

In [20]:
# error
# `a` in left is datetime type, but in right is string.

# left.merge(right, on=["datetime"])

In [21]:
# error
# the type of keys in left isn't same as the right keys.

# left.merge(right, on=["a", "datetime"])

In [22]:
right["a"] = right["a"].astype(int)
left.merge(right, on=["a"])

,a,b,datetime_x,c,datetime_y
0,1,O,2021-01-01,u,2021-01-03
1,3,L,2021-01-02,t,2021-01-04
2,5,k,2021-01-03,V,2021-01-05
3,7,Z,2021-01-05,x,2021-01-06


In [23]:
right["datetime"] = pd.to_datetime(right["datetime"])
left.merge(right, on=["datetime"])

,a_x,b,datetime,a_y,c
0,5,k,2021-01-03,1,u
1,6,F,2021-01-04,3,t
2,7,Z,2021-01-05,5,V


整型与浮点型关联时的情况：

In [24]:
right["a"] = right["a"].astype(float)
right.head()

,a,c,datetime
0,1.0,u,2021-01-03
1,3.0,t,2021-01-04
2,5.0,V,2021-01-05
3,7.0,x,2021-01-06
4,9.0,m,2021-01-07


In [25]:
left.merge(right, on=["a"])

,a,b,datetime_x,c,datetime_y
0,1,O,2021-01-01,u,2021-01-03
1,3,L,2021-01-02,t,2021-01-04
2,5,k,2021-01-03,V,2021-01-05
3,7,Z,2021-01-05,x,2021-01-06


In [26]:
left.merge(right, on=["a"]).dtypes

a                      int64
b                     object
datetime_x    datetime64[ns]
c                     object
datetime_y    datetime64[ns]
dtype: object

In [27]:
right["a"] = [1.0, 3.1, 5.0, 7.4, 9.5]
right.head()

,a,c,datetime
0,1.0,u,2021-01-03
1,3.1,t,2021-01-04
2,5.0,V,2021-01-05
3,7.4,x,2021-01-06
4,9.5,m,2021-01-07


In [28]:
left.merge(right, on=["a"])

/Users/Bobot/.virtualenvs/sspai-100-hours-series-python/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1215: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(


,a,b,datetime_x,c,datetime_y
0,1,O,2021-01-01,u,2021-01-03
1,5,k,2021-01-03,V,2021-01-05


In [29]:
left.merge(right, on=["datetime"])

,a_x,b,datetime,a_y,c
0,5,k,2021-01-03,1.0,u
1,6,F,2021-01-04,3.1,t
2,7,Z,2021-01-05,5.0,V


In [30]:
left.merge(right, on=["a"], suffixes=("", "_right"))

/Users/Bobot/.virtualenvs/sspai-100-hours-series-python/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1215: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  warnings.warn(


,a,b,datetime,c,datetime_right
0,1,O,2021-01-01,u,2021-01-03
1,5,k,2021-01-03,V,2021-01-05


In [31]:
left.merge(right.drop("a", axis=1), on=["datetime"])

,a,b,datetime,c
0,5,k,2021-01-03,u
1,6,F,2021-01-04,t
2,7,Z,2021-01-05,V


## join

In [32]:
left = pd.DataFrame(
    {
        "id": [1, 3, 3, 6, 7],
        "letter_left": random.sample(ascii_letters, k=5),
    }
).set_index("id")
left.head()

,letter_left
id,
1,r
3,a
3,q
6,e
7,A


In [33]:
right = pd.DataFrame(
    {
        "id": ["1", "3", "5", "7", "9"],
        "letter_right": random.choices(ascii_letters, k=5),
    }
)

right.head()

,id,letter_right
0,1,J
1,3,v
2,5,e
3,7,r
4,9,t


In [34]:
# default left join

left.join(right)

,letter_left,id,letter_right
1,r,3,v
3,a,7,r
3,q,7,r
6,e,NaN,NaN
7,A,NaN,NaN


In [35]:
left.join(right, how="left")

,letter_left,id,letter_right
1,r,3,v
3,a,7,r
3,q,7,r
6,e,NaN,NaN
7,A,NaN,NaN


In [36]:
left = left.rename(columns={"letter_left": "letter"})
left.head()

,letter
id,
1,r
3,a
3,q
6,e
7,A


In [37]:
right = right.rename(columns={"letter_right": "letter"})
right.head()

,id,letter
0,1,J
1,3,v
2,5,e
3,7,r
4,9,t


In [38]:
left.join(right, how="inner", lsuffix="_left", rsuffix="_right")

,letter_left,id,letter_right
1,r,3,v
3,a,7,r
3,q,7,r
